# Homework 5 Exercise 1

## Lab Instructions
All your answers for Exercise 1 should be written in this notebook. You shouldn't need to write or modify any other files.

**You should execute every block of code to not miss any dependency.**

*This exercise was developed by Maximilian Hüttenrauch for the KIT Cognitive Systems Lecture, June 2020.
  It is adapted from previous Robot Learning Lectures at TU Darmstadt, the Berkeley Deep RL Bootcamp
  [Lab1](https://sites.google.com/view/deep-rl-bootcamp/home)
  and Berkeley Deep RL Class
   [HW2](https://github.com/berkeleydeeprlcourse/homework/blob/c1027d83cd542e67ebed982d44666e0d22a00141/hw2/HW2.ipynb)
   [(license)](https://github.com/berkeleydeeprlcourse/homework/blob/master/LICENSE)*

In [ ]:
from ex1.mdp import MDP
from ex1.misc import make_grader
from ex1.misc import plot_v_pi
from ex1.misc import expected_output_pi_infinite
import numpy as np

### MDP

The MDP class for the cleaning robot is already implemented and doesn't need to be changed.

In [ ]:
# Actions
DOWN = 0
RIGHT = 1
UP = 2
LEFT = 3
STAY = 4


class RoboMDP(MDP):
    O = -1e3  # Obstacles, walls or inaccessible places
    D = 2  # Dirt
    W = -1  # Water
    C = -30  # Cat
    T = 30  # Toy
    E = 0  # Empty

    grid_world = {
        "reward": [
            E, O, O, E, E, O, O, E, E, E,
            E, E, E, E, D, O, E, E, D, E,
            E, D, E, E, E, O, E, E, O, E,
            O, O, O, O, E, O, E, O, O, O,
            D, E, E, D, E, O, T, D, E, E,
            E, O, D, D, E, O, W, E, E, E,
            W, O, E, O, E, O, D, O, O, E,
            W, E, E, O, D, E, E, O, D, E,
            E, E, E, D, C, O, E, E, D, E
        ],
        "map": [
            " OO  OO   ",
            "    DO  D ",
            " D   O  O ",
            "OOOO O OOO",
            "D  D OTD  ",
            " ODD OW   ",
            "WO O ODOO ",
            "W  OD  OD ",
            "   DCO  D "
        ]
    }

    n_actions = 5
    world_dim = (9, 10)

    # Transition probabilities. Each row corresponds to one action, i.e. choosing action down has a 70% chance of
    # executing correctly, and 10% chance for executing right, left or stay.
    # Usage: self.tp[actual_action, noisy_executed_action]

    #             down, right, up, left, stay
    tp = np.array([[0.7, 0.1, 0.0, 0.1, 0.1],  # down
                  [0.1, 0.7, 0.1, 0.0, 0.1],  # right
                  [0.0, 0.1, 0.7, 0.1, 0.1],  # up
                  [0.1, 0.0, 0.1, 0.7, 0.1],  # left
                  [0.0, 0.0, 0.0, 0.0, 1.0]])  # stay

    def __init__(self, discount: float, stochastic: bool):
        super(RoboMDP, self).__init__(discount, self.world_dim, self.n_actions, desc=self.grid_world['map'])

        def inc(row: int, col: int, action: int):
            if action == DOWN:  # move down
                row = min(row+1, self.world_dim[0] - 1)
            elif action == RIGHT:  # move right
                col = min(col+1, self.world_dim[1] - 1)
            elif action == UP:  # move up
                row = max(row - 1, 0)
            elif action == LEFT:  # move left
                col = max(col - 1, 0)
            elif action == STAY:  # stay
                pass  # Not moving
            return row, col

        # We need to set the next state with its corresponding transition probability based on the current state and
        # action. Have a look at mdp.py for the implementation of self.stp
        for r in range(self.world_dim[0]):
            for c in range(self.world_dim[1]):
                s = self.to_s(r, c)
                self.rewards[s] = self.grid_world["reward"][s]
                for a in range(self.n_actions):
                    stp_list = self.stp[s, a]
                    if stochastic:
                        for noisy_a in range(self.n_actions):
                            if self.tp[a, noisy_a] > 0:
                                next_r, next_c = inc(r, c, noisy_a)
                                next_state = self.to_s(next_r, next_c)
                                stp_list.append((next_state, self.tp[a, noisy_a]))
                    else:
                        next_r, next_c = inc(r, c, a)
                        next_state = self.to_s(next_r, next_c)
                        stp_list.append((next_state, 1.))

    # helper functions

    def to_s(self, row, col):
        """given a row and column, returns a linear index corresponding to the world dimension"""
        return row * self.world_dim[1] + col

    def to_rc(self, s):
        """given a linear state, returns a row and column corresponding to the world dimension"""
        return np.unravel_index(s, self.world_dim)


## Policy Iteration

For this exercise, you'll implement policy iteration, which has the following pseudocode:

---
Initialize $V_{(0)}^{\pi_0}(s)=0$ for all $s$, $\pi_0 \leftarrow$ uniform, $k = 0$

* For $i=1, 2, \dots$

    If $i>0$: Initialize $V_{(0)}^{\pi_{i}}(s) = V_{(k)}^{\pi_{i-1}}(s)$ (i.e. initialize the value function of the new policy with the converged value function of the old policy)

    * For $k=0, 1, 2, \dots$

    $V_{(k+1)}^{\pi_{i}}(s) = \sum_a \pi(a \mid s)  \Big( r(s,a) + \gamma \sum_{s'} P(s' \mid s,a) \, V_{(k)}^{\pi_{i}}(s') \Big)$

    Until convergence

 $$\pi_{(i+1)}(s) = \begin{cases}1, & \underset{a}{\arg \max} \Big( r(s,a) + \gamma \sum_{s'} P(s' \mid s,a) V^{\pi_i}(s')\Big)\\0 & else\end{cases}$$

Until convergence

---

Your code will return two lists: $[V_{(0)}, V_{(1)}, \dots, V_{(n)}]$ and $[\pi_{(0)}, \pi_{(1)}, \dots, \pi_{(n-1)}]$

To ensure that you get the same policies as the reference solution, choose the lower-index action to break ties in $\arg \max_a$. This is done automatically by np.argmax. This will only affect the "# chg actions" printout below--it won't affect the values computed.

### Hints
- We separately implement policy evaluation and policy improvement
- The reward function can be accessed by $\texttt{mdp.rewards[s]}$, the next states and their probability of reaching
 them (given a state $s$ and and action $a$) by $\texttt{mdp.stp[s, a]}$
- Use the functions $\texttt{to_s}$ and $\texttt{to_rc}$ provided by the MDP class to convert between linear states
 $s \in [0, |\mathcal{S}|]$ and row/column format, i.e. $\texttt{mdp.to_rc(0)}$ returns $(0, 0)$, while
 $\texttt{mdp.to_s(0, 0)}$ returns $0$
- For your implementation, the identities $V^\ast(s) = \max_a Q^\ast(s, a)$ and
 $\pi^\ast(s) = \underset{a}{\arg \max} \, Q^\ast(s, a)$ may be helpful
- Follwing numpy functions may be useful: $\texttt{np.sum}$, $\texttt{np.max}$, $\texttt{np.argmax}$
- Try using list comprehensions as this will shorten your code
- We provide a printing function $\texttt{make_grader}$ which compares the output of your implementation with the output
of a correct implementation of value iteration. For each iteration, it prints the largest difference of the value function,
 the number of actions that have changed from the last iteration, as well as the value of state 0.
 If your solution is correct, it will print $\textit{Test succeeded}$

### 1 a) Implementing policy evaluation

In [ ]:
def policy_evaluation(mdp, pi_prob, v=None):
    """
    Inputs:
    @param mdp: the mdp object
    @param pi_prob: action probabilities
    @param v: initial value function (optional)
    Outputs:
    @return: value function of the provided policy
    """

    if v is None:
        # Initialize value function for policy evaluation
        v = np.zeros(shape=mdp.world_dim)
    for pe_iter in range(10000):
        # save current estimate
        v_prev = np.copy(v)

        # %%%%%%%% add your code here %%%%%%%%%%%

        ...

        # %%%%%%%% add your code here %%%%%%%%%%%

        # run policy evaluation until convergence
        if np.allclose(v, v_prev):
            break

    return v

### 1 b) Implementing policy improvement

In [ ]:
def policy_improvement(mdp, v):
    """
    Inputs:
    @param mdp: the mdp object
    @param v: value function of a policy
    Outputs:
    @return: new policy and distribution
    """

    # initialize policy
    pi = np.zeros(shape=mdp.world_dim)
    pi_prob = np.ones(shape=mdp.world_dim + (mdp.n_actions,))

    # %%%%%%%% add your code here %%%%%%%%%%%

    ...

    # %%%%%%%% add your code here %%%%%%%%%%%

    return pi, pi_prob

In [ ]:
def policy_iteration(mdp, n_iter, grade_print=print):
    """
    Inputs
    @param mdp: an MDP object
    @param n_iter: number of PI iterations
    @param grade_print: a printing function
    Outputs
    @return: (value_functions, policies)

    len(value_functions) == n_iter+1 and len(policies) == n_iter
    """
    vs = [np.zeros(shape=mdp.world_dim)]
    pis = []  # contains the actual actions
    pi_prob = np.ones(shape=mdp.world_dim + (mdp.n_actions, )) / mdp.n_actions  # contains the action probabilities for each state

    grade_print("Iteration |  max|V-Vprev|  | # chg actions | V[0]    ")
    grade_print("----------+----------------+---------------+---------")
    for pi_iter in range(n_iter):
        pi_old = pis[-1] if len(pis) > 0 else None  # \pi^{(it)} = Greedy[V^{(it-1)}]. Just used for printout
        v_old = vs[-1]

        # run policy evaluation
        v_init = np.copy(v_old)
        v = policy_evaluation(mdp, pi_prob, v_init)

        # run policy improvement
        pi, pi_prob = policy_improvement(mdp, v)

        max_diff = np.abs(v - v_old).max()

        n_chg_actions = 0 if pi_old is None else (pi != pi_old).sum()
        grade_print("{:4d}      | {:12.5f}   |   {:4d}        | {:8.3f}".format(pi_iter, max_diff, n_chg_actions, v[0, 0]))

        vs.append(v)
        pis.append(pi)

        if max_diff < 1e-5:
            break

    return vs, pis

## 1 c) Infinite Horizon Problem

In [ ]:
robo_mdp_infinite = RoboMDP(discount=0.8, stochastic=False)
v_infinite, pi_infinite = policy_iteration(robo_mdp_infinite, 100, make_grader(expected_output_pi_infinite))

plot_v_pi(v_infinite[-1], pi_infinite[-1], robo_mdp_infinite)

In [ ]:
# Put your Text answers here

## 1 d) Policy Evaluation

In [ ]:
# Put your Text answers here